In [1]:
import numpy as np
import random
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import copy
import matplotlib.pyplot as plt

In [39]:
from google.colab import files
uploaded = files.upload()

Saving problem_2.npz to problem_2.npz


In [40]:
problem = np.load('problem_2.npz')
x = problem['x']
y = problem['y']
x.shape

(3, 5000)

In [5]:
def safe_sqrt(x):
    return np.sqrt(np.abs(x))
safe_sqrt.__name__ = "sqrt(abs)"

def safe_log(x):
    return np.log(np.abs(x) + 1e-9)
safe_log.__name__ = "log(abs)"

def safe_log10(x):
    return np.log10(np.abs(x) + 1e-9)
safe_log10.__name__ = "log10(abs)"


np_functions = [np.add, np.subtract, np.multiply, np.divide,np.maximum,np.minimum, np.power]
np_functions_1D=[safe_log, safe_log10, np.exp, np.sin, np.cos, np.tan, safe_sqrt,np.sign, np.square]

class Node:
    def __init__(self, a, children=[], action=None, result=None, leaf=False, text=''):
        self.x=a
        self.action = action
        self.children=children
        self.result = result
        self.text = text
        self.leaf=leaf
        if self.result is None:
            if leaf==True:
                if random.random()<0.2:
                    self.result=round(random.uniform(-50, 50),1)
                    self.text=str(self.result)
                else:
                    row_indice=random.choices(range(self.x.shape[0]), k=1)
                    self.result=self.x[row_indice[0]]
                    self.text='x'+str(row_indice)+''
            else:

                if len(self.children)==2:
                    if self.action is None:
                        self.action=random.choice(np_functions)

                    self.text='('+self.children[0].text+' '+self.action.__name__+' '+self.children[1].text+')'
                    self.result=self.action(self.children[0].result,self.children[1].result)
                else:
                    if self.action is None:
                        self.action=random.choice(np_functions_1D)
                    self.text=self.action.__name__+'('+self.children[0].text+')'
                    self.result=self.action(self.children[0].result)

    def update(self):
        if self.leaf==False:
            if len(self.children)==2:
                    self.text='('+self.children[0].text+' '+self.action.__name__+' '+self.children[1].text+')'
                    self.result=self.action(self.children[0].result,self.children[1].result)
            else:
                    self.text=self.action.__name__+'('+self.children[0].text+')'
                    self.result=self.action(self.children[0].result)


    def action_in_tree(self, function, child_node_pourcentage=0.5):
        if self.leaf==False:
            if len(self.children) == 2:
                i = random.randint(0, 1)
                if self.children[i].leaf == False and random.random() <= child_node_pourcentage:
                    self.children[i].action_in_tree(function, child_node_pourcentage)
                    self.update()
                elif self.children[1 - i].leaf == False and random.random() <= child_node_pourcentage:
                    self.children[1 - i].action_in_tree(function, child_node_pourcentage)
                    self.update()
                else:
                    function(self)
            elif len(self.children) == 1:
                if self.children[0].leaf == False and random.random() <= child_node_pourcentage:
                    self.children[0].action_in_tree(function, child_node_pourcentage)
                else:
                    function(self)

    def print_text(self):
        self.update()
        print(self.text)

    def change_action(self):
        if len(self.children)==2:
            self.action=random.choice(np_functions)
        else:
            self.action=random.choice(np_functions_1D)
        self.update()

    def create_leaf(self):
        self.leaf=True
        if random.random()<0.15:
            self.result=round(random.uniform(-50, 50),1)
            self.text=str(self.result)
        else:
            row_indice=random.choices(range(self.x.shape[0]), k=1)
            self.result=self.x[row_indice[0]]
            self.text='x'+str(row_indice)+''

    def cut_branch(self):
        if self.leaf==False:
            if len(self.children)==2:
                i=random.randint(0, 1)
                if self.children[i].leaf==False:
                    self.children[i].cut_branch()
                    self.update()
                elif self.children[1-i].leaf==False:
                    self.children[1-i].cut_branch()
                    self.update()
                else:
                    self.children=[]
                    self.create_leaf()
            else:
                if self.children[0].leaf==False:
                    self.children[0].cut_branch()
                    self.update()
                else:
                    self.children=[]
                    self.create_leaf()

    def expand_leaf(self):
        if self.leaf:
            self.children=[copy.deepcopy(self)]
            self.leaf=False
            for i in range(random.randint(0, 1)):
                self.children.append(Node(self.x, leaf=True))
            if len(self.children)==2:
                self.action=random.choice(np_functions)
            else:
                self.action=random.choice(np_functions_1D)
        else:
            if len(self.children)>0:
                random.choice(self.children).expand_leaf()
        self.update()

    def modify_leaf_number(self):
          if self.leaf and  "x" not in self.text:
              operation=random.choice(['multiply', 'divide'])
              while True:
                  factor = round(random.uniform(-1, 10), 1)
                  if factor != 0:
                      break
              if operation == 'multiply':
                  self.result*= factor
              elif operation == 'divide':
                  self.result=self.result/factor

              self.text = str(self.result)
              return True
          elif len(self.children)>0:

              shuffled_children=random.sample(self.children, len(self.children))
              for child in shuffled_children:
                  if child.modify_leaf_number():
                      self.update()
                      return True
          return False


    def depth(self):
        """
        Return the depth of the tree
        """
        if self.leaf:
            return 1
        return 1 + max(child.depth() for child in self.children)

    def random_branch(self,max_depth=5):
        """
        Cut a branch from the tree and return this branch (it can be the all tree).
        In reallity seing the construction of the tree this method return a random node from the tree.
        """
        if self.leaf:
            return self
        else:
            child=random.choice(self.children)
            if random.random()<(1/self.depth()) and self.depth()<max_depth: #The more we go closer to the end of the tree the more chance there is of a cut.
                return self
            else:
                return child.random_branch()

    def MSE(self,y):
        self.result=np.nan_to_num(self.result, nan=0.0)
        if not isinstance(self.result, np.ndarray):
            self.result =np.full_like(y, self.result, dtype=float)
        return mean_squared_error(self.result,y)

    def MAE(self,y):
        self.result=np.nan_to_num(self.result, nan=0.0)
        if not isinstance(self.result, np.ndarray):
            self.result =np.full_like(y, self.result, dtype=float)
        return mean_absolute_error(self.result,y)

    def R2(self,y):
        if not isinstance(self.result, np.ndarray):
            self.result = np.array([self.result])
        return r2_score(self.result,y)

    def calculate_alpha_beta(self, y):
        """
        Calculate and return the scalar alpha (covariance(y,self)/variance(self)) and beta (mean(y)-mean(self))
        This parameter recenter the result to have the same mean and variance as y.
        """
        result = copy.deepcopy(self.result)
        g_x = result.flatten()
        y_flat = y.flatten()
        g_x = np.nan_to_num(g_x, nan=0.0, posinf=0.0, neginf=0.0) #Check for any infiny or nan value and replace it by 0.
        y_flat = np.nan_to_num(y_flat, nan=0.0, posinf=0.0, neginf=0.0)
        covariance=np.mean((g_x-np.mean(g_x))*(y_flat-np.mean(y_flat)))
        variance=np.mean((g_x-np.mean(g_x))**2)
        alpha=covariance/variance if variance!=0 else variance
        beta=np.mean(y_flat)-alpha*np.mean(g_x)
        return alpha,beta

    def MSE_flexible(self, y):
        self.result = np.nan_to_num(self.result, nan=0.0, posinf=0.0, neginf=0.0)
        y = np.array(y).flatten()
        if not isinstance(self.result, np.ndarray):
            self.result = np.full_like(y, self.result, dtype=float)
        else:
            self.result = self.result.flatten()
        y = np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)
        alpha, beta = self.calculate_alpha_beta(y)
        predictions=alpha*self.result+beta #Calculate the final MSE with alpha and beta calibrations
        predictions=np.nan_to_num(predictions, nan=0.0, posinf=0.0, neginf=0.0)
        return mean_squared_error(y, predictions)



In [6]:
def encapsule_node(node):
    return Node(node.x,children=[node])

In [7]:
def create_random_tree(a, depth=4):
    if depth == 0 or random.random() < 0.3:
        return Node(a,leaf=True)
    if random.random()<0.7:
        left_child = create_random_tree(a,depth - 1)
        right_child = create_random_tree(a,depth - 1)
        return Node(x,children=[left_child,right_child])
    else:
        child = create_random_tree(a,depth - 1)
        return Node(x,children=[child])

In [8]:
def select_best_tree(a, y_target, num_trees=10000, max_depth=3):
    best_tree = None
    best_mse = float('inf')

    for _ in range(num_trees):
        tree = create_random_tree(a, depth=max_depth)

        mse = tree.MSE( y_target)
        if mse < best_mse:
            best_tree = tree
            best_mse = mse

    return best_tree, best_mse

In [9]:
def create_neighbor(tree, max_depth=9,actions=["cut_branch", "expand_leaf","change_action","modify_leaf_number","encapsulate"],weights=[1,1,0.5,0.2,1]):
  #I tried with different recurence of mutation (some take more time to compute like modify_leaf because they need to go through the all tree so I tried to limit them.)
    neighbor = copy.deepcopy(tree)
    if tree.depth()>max_depth:
        weights[actions.index("expand_leaf")] = 0
        weights[actions.index("encapsulate")] = 0
    if tree.leaf==True:
        weights[actions.index("cut_branch")] = 0
        weights[actions.index("change_action")] = 0
    action = random.choices(actions, weights=weights, k=1)[0]
    if action == "cut_branch":
        neighbor.cut_branch()
    elif action == "encapsulate":
        neighbor=encapsule_node(neighbor)
    elif action == "expand_leaf":
        neighbor.expand_leaf()
    elif action == "change_action":
        neighbor.action_in_tree(Node.change_action)
    elif action =="modify_leaf_number":
        if not neighbor.modify_leaf_number():
            create_neighbor(tree,actions=["cut_branch", "expand_leaf","encapsulate"],weights=[1,1,0.5])
    return neighbor

In [10]:
class Tree_Problem:
    def __init__(self, a,b):
        self.a=a
        self.b=b
        self.tree=create_random_tree(self.a)
    def random_solution(self):
        return create_random_tree(self.a)
    def value(self,tree):
        return tree.MSE(self.b)

    def neighbour(self, tree):
        return create_neighbor(tree)

# Simulated Annealing

In [ ]:
def exponential_schedule(temperature=1000, alpha=0.8, limit=100):#This function describe the evolution of the temperature
    return lambda t: (temperature * np.exp(-alpha * t) if t < limit else 1e-3)

def simulated_annealing(problem, max_iterations=50,schedule=exponential_schedule(), max_evaluations=1000,current_solution=problem.random_solution()):
    # Initialisation de la solution
    current_solution = current_solution
    current_value = problem.value(current_solution)
    best_solution = current_solution
    best_value = current_value
    num_evaluations = 1
    costs = [current_value]
    temperature =schedule(0) #Initial temperature
    t=1
    fit_list=[]
    while num_evaluations < max_evaluations and temperature >0:
        neighbour_solution=problem.neighbour(current_solution)
        neighbour_value=problem.value(neighbour_solution)
        num_evaluations+=1
        fit_list.append([neighbour_solution,neighbour_value])
        costs.append(neighbour_value)

        if neighbour_value<current_value:
            current_solution=neighbour_solution
            current_value=neighbour_value
            if current_value<best_value:
                best_solution = current_solution
                best_value = current_value

        else:
            probability = np.exp((current_value-neighbour_value)/temperature)
            if random.random()<probability:
                current_solution=neighbour_solution
                current_value=neighbour_value

        #We calculate the temperature (cooling it down)
        temperature=schedule(t)
        t+=1# Increment time step


    return best_solution, best_value, num_evaluations, costs, fit_list


# Genetic Algorithm

In [11]:
def init_population(problem,population_size):
    pop=[]
    for i in range(population_size):
        pop.append(create_random_tree(problem.a))
    return pop

## Using MSE

In [ ]:
#combines elitism and rank selections.
#Rank selection in GA chooses individuals based on their relative performance rather than their actual fitness scores.
#Elitism directly selects the best individuals (elites) based on their fitness (price).
def select_elitismRank(problem, population,n_population,elite_size):#n_population is number of individuals wanted
    #and elite_size is the ratio of elite values in the said population
    # Select best roll (elites) directly
    n_elites = int(elite_size * n_population)
    elites = sorted(population, key=lambda tree: tree.MSE(problem.b), reverse=False)[:n_elites]
    #First we are sorting the values from the worst fitness to the best
    ranked_population = sorted(population, key=lambda tree: tree.MSE(problem.b), reverse=False)
    #then we are creating the weigth going from 1 for the worst fitness and n_population for the best
    weights = range(1, len(population)+1)
    selected = random.choices(ranked_population, weights, k=n_population - n_elites)
    return elites + selected

In [ ]:
def genetic_algorithm(problem, population_size=100, generations=70, elite_size=0.2, selection_func=select_elitismRank):
    # Initialize the population
    population = init_population(problem,population_size)
    best_tree = min(population, key=lambda tree: tree.MSE(problem.b))

    best_MSE = best_tree.MSE(problem.b)
    print(best_MSE)
    fit_list=[]
    no_improvement_streak = 0
    # Evolution loop
    for _ in range(generations):
        new_population = []
        # Apply crossover and mutation
        for i in range(population_size):
            parents = selection_func(problem, population,15,elite_size)
            parent1,parent2=random.sample(parents,2)
            child = Node(problem.a, children=[parent1.random_branch(),parent2.random_branch()])
            child = create_neighbor(child)
            new_population.append(child)

        # Replace old population with new population
        population = new_population
        current_best_tree =  min(new_population, key=lambda tree: tree.MSE(problem.b))
        current_best_MSE = current_best_tree.MSE(problem.b)
        print(current_best_MSE)
        if current_best_MSE < best_MSE:
            best_MSE = current_best_MSE
            best_tree = current_best_tree
            no_improvement_streak = 0
        else:
            no_improvement_streak += 1
        print(no_improvement_streak)
        if no_improvement_streak > 15:
            population = init_population(problem,population_size)
            break
        print(best_MSE)
        fit_list.append(best_MSE)
    return best_tree, best_MSE, fit_list,child, population



## Using MSE flexible

In [12]:
#combines elitism and rank selections.
#Rank selection in GA chooses individuals based on their relative performance rather than their actual fitness scores.
#Elitism directly selects the best individuals (elites) based on their fitness (price).
def select_elitismRank(problem, population,n_population,elite_size):#n_population is number of individuals wanted
    #and elite_size is the ratio of elite values in the said population
    # Select best roll (elites) directly
    n_elites = int(elite_size * n_population)
    elites = sorted(population, key=lambda tree: tree.MSE_flexible(problem.b), reverse=False)[:n_elites]
    #First we are sorting the values from the worst fitness to the best
    ranked_population = sorted(population, key=lambda tree: tree.MSE_flexible(problem.b), reverse=False)
    #then we are creating the weigth going from 1 for the worst fitness and n_population for the best
    weights = range(1, len(population)+1)
    selected = random.choices(ranked_population, weights, k=n_population - n_elites)
    return elites + selected

In [13]:
def genetic_algorithm(problem, population_size=60, generations=120, elite_size=0.2, selection_func=select_elitismRank):
    # Initialize the population
    population = init_population(problem,population_size)
    best_tree = min(population, key=lambda tree: tree.MSE_flexible(problem.b))

    best_MSE = best_tree.MSE_flexible(problem.b)
    print(best_MSE)
    fit_list=[]
    no_improvement_streak = 0
    # Evolution loop
    for _ in range(generations):
        new_population = []
        # Apply crossover and mutation
        for i in range(population_size):
            parents = selection_func(problem, population,10,elite_size)
            parent1,parent2=random.sample(parents,2)
            child = Node(problem.a, children=[parent1.random_branch(),parent2.random_branch()])
            child = create_neighbor(child)
            new_population.append(child)

        # Replace old population with new population
        population = new_population
        current_best_tree =  min(new_population, key=lambda tree: tree.MSE_flexible(problem.b))
        current_best_MSE = current_best_tree.MSE_flexible(problem.b)
        print(current_best_MSE)
        if current_best_MSE < best_MSE:
            best_MSE = current_best_MSE
            best_tree = current_best_tree
            no_improvement_streak = 0
        else:
            no_improvement_streak += 1
        print(no_improvement_streak)
        if no_improvement_streak > 10:
            num_to_cull = int(population_size * 0.9)
            population = sorted(population, key=lambda tree: tree.MSE_flexible(problem.b))[:-num_to_cull]
            new_individuals = init_population(problem, num_to_cull)
            population+=new_individuals
            no_improvement_streak = 0
        print(best_MSE)
        print(best_tree.text)
        fit_list.append(best_MSE)
    return best_tree, best_MSE, fit_list,child, population

